In [38]:
!pip install -U googlemaps

In [39]:
!pip install mpu

In [40]:
import googlemaps
from datetime import datetime
import os
import yaml
import pandas as pd
import numpy as np
from itertools import combinations
import mpu

In [41]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [42]:
gmaps = googlemaps.Client(key=PARAM["GOOGLE_API_KEY"])

In [43]:

geocode_result = gmaps.geocode('Hohes Feld 4, 38124 Braunschweig, Germany')
print (geocode_result[0]["geometry"]["location"]["lat"], geocode_result[0]["geometry"]["location"]["lng"])

52.2106571 10.5232126


In [44]:
addresses = pd.read_csv("address.tsv", sep="\t")

In [45]:
addresses

,name,address
0,A,"Hohes Feld 4, 38124 Braunschweig, Germany"
1,B,"Leipziger Str. 220, 38124 Braunschweig, Germany"
2,C,"Hohes Feld 4, 31226 Peine, Germany"
3,D,"Hohes Feld 2A, 38124 Braunschweig, Germany"


In [46]:
for index, row in addresses.iterrows():
    geocode_result = gmaps.geocode(row["address"]) 
    lat = geocode_result[0]["geometry"]["location"]["lat"]
    lng = geocode_result[0]["geometry"]["location"]["lng"]

    addresses.loc[index, "lat"] = lat
    addresses.loc[index, "lng"] = lng

In [47]:
addresses

,name,address,lat,lng
0,A,"Hohes Feld 4, 38124 Braunschweig, Germany",52.210657,10.523213
1,B,"Leipziger Str. 220, 38124 Braunschweig, Germany",52.210868,10.523072
2,C,"Hohes Feld 4, 31226 Peine, Germany",52.293380,10.215091
3,D,"Hohes Feld 2A, 38124 Braunschweig, Germany",52.210207,10.521785


In [48]:
idxs = list(combinations(addresses.index,2))
dfx = pd.DataFrame(np.concatenate((addresses.iloc[idx[0]].values, addresses.iloc[idx[1]].values)) for idx in idxs)


In [49]:
dfx.columns = ['s_name', 's_address', 's_lat', 's_lng', 'd_name', 'd_address', 'd_lat', 'd_lng']

In [50]:
dfx

,s_name,s_address,s_lat,s_lng,d_name,d_address,d_lat,d_lng
0,A,"Hohes Feld 4, 38124 Braunschweig, Germany",52.210657,10.523213,B,"Leipziger Str. 220, 38124 Braunschweig, Germany",52.210868,10.523072
1,A,"Hohes Feld 4, 38124 Braunschweig, Germany",52.210657,10.523213,C,"Hohes Feld 4, 31226 Peine, Germany",52.293380,10.215091
2,A,"Hohes Feld 4, 38124 Braunschweig, Germany",52.210657,10.523213,D,"Hohes Feld 2A, 38124 Braunschweig, Germany",52.210207,10.521785
3,B,"Leipziger Str. 220, 38124 Braunschweig, Germany",52.210868,10.523072,C,"Hohes Feld 4, 31226 Peine, Germany",52.293380,10.215091
4,B,"Leipziger Str. 220, 38124 Braunschweig, Germany",52.210868,10.523072,D,"Hohes Feld 2A, 38124 Braunschweig, Germany",52.210207,10.521785
5,C,"Hohes Feld 4, 31226 Peine, Germany",52.293380,10.215091,D,"Hohes Feld 2A, 38124 Braunschweig, Germany",52.210207,10.521785


In [51]:
for index, row in dfx.iterrows():
    dist = mpu.haversine_distance((row["s_lat"], row["s_lng"]), (row["d_lat"], row["d_lng"]))
    dfx.loc[index, "distance"] = dist

In [52]:
dfx

,s_name,s_address,s_lat,s_lng,d_name,d_address,d_lat,d_lng,distance
0,A,"Hohes Feld 4, 38124 Braunschweig, Germany",52.210657,10.523213,B,"Leipziger Str. 220, 38124 Braunschweig, Germany",52.210868,10.523072,0.025376
1,A,"Hohes Feld 4, 38124 Braunschweig, Germany",52.210657,10.523213,C,"Hohes Feld 4, 31226 Peine, Germany",52.293380,10.215091,22.902873
2,A,"Hohes Feld 4, 38124 Braunschweig, Germany",52.210657,10.523213,D,"Hohes Feld 2A, 38124 Braunschweig, Germany",52.210207,10.521785,0.109389
3,B,"Leipziger Str. 220, 38124 Braunschweig, Germany",52.210868,10.523072,C,"Hohes Feld 4, 31226 Peine, Germany",52.293380,10.215091,22.884647
4,B,"Leipziger Str. 220, 38124 Braunschweig, Germany",52.210868,10.523072,D,"Hohes Feld 2A, 38124 Braunschweig, Germany",52.210207,10.521785,0.114477
5,C,"Hohes Feld 4, 31226 Peine, Germany",52.293380,10.215091,D,"Hohes Feld 2A, 38124 Braunschweig, Germany",52.210207,10.521785,22.834263


In [53]:
dfx[dfx["distance"] < 0.05]

,s_name,s_address,s_lat,s_lng,d_name,d_address,d_lat,d_lng,distance
0,A,"Hohes Feld 4, 38124 Braunschweig, Germany",52.210657,10.523213,B,"Leipziger Str. 220, 38124 Braunschweig, Germany",52.210868,10.523072,0.025376
